# **Inferring**
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
import openai
from openai import OpenAI
with open('key.txt', 'r') as file:
    OPENAI_API_KEY = file.read().strip()

In [4]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Product review text

In [5]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [6]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is positive. The reviewer is satisfied with the lamp, the customer service, and the company in general.


In [7]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [8]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Identify anger

In [9]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [10]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [11]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.

In [12]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

Five topics discussed in the article are requested from the model in a format that each item is one or two words long and in a comma-separated list. ChatGPT returns the topics as government surveys, job satisfaction, NASA, etc.

In [13]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas without numbering them.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Government survey, Employee satisfaction, NASA, Social Security Administration, Job satisfaction


In [14]:
response.split(sep=', ')

['Government survey',
 'Employee satisfaction',
 'NASA',
 'Social Security Administration',
 'Job satisfaction']

## Make a news alert for certain topics

The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:

In [15]:
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

In [16]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a dictionay where the key is a topic and the value is 0 or 1 for each topic if it appears.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [17]:
for i in response.split(', '):
    print(i)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [18]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep=', ')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ValueError: invalid literal for int() with base 10: '1,\n    "local government"'

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [19]:
sandwitch = """I had high hopes for this sandwich, but it was an absolute disappointment. First off, the bread was stale and dry, making every bite feel like chewing on cardboard. It completely overpowered the rest of the ingredients, which were already lacking in both flavor and freshness.

The filling was just as bad—barely any meat, a sad excuse for cheese, and a few limp, flavorless veggies that looked like they had been sitting out for hours. The so-called “special sauce” was practically non-existent, and the whole sandwich was bland and lifeless.

To make matters worse, it was cold and soggy, as if it had been made ages ago and left to sit. The worst part? It was ridiculously overpriced for something that tasted this bad. I could have made a better sandwich at home with half the effort and half the cost.

Overall, a huge letdown. I won’t be ordering this again. """

In [20]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{sandwitch}'''
"""
response = get_completion(prompt)
print(response)

negative


In [21]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{sandwitch}'''
"""
response = get_completion(prompt)
print(response)

disappointment, frustration, dissatisfaction, anger, regret


In [22]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{sandwitch}'''
"""
response = get_completion(prompt)
print(response)

Yes


In [23]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{sandwitch}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "sandwich",
  "Brand": "unknown"
}


In [ ]:
#I used the openai API for sentiment analysis, I evaluated its ability to detect emotions in various reviews accurately.

#GPT effectively classified sentiments as positive, neutral, or negative, demonstrating strong contextual understanding. It performed well in clear-cut cases but struggled with nuanced emotions or sarcasm.